In [37]:
import torch
import jax
import jax.numpy as jnp
from jax import random
from flax.serialization import from_bytes
import sys
sys.path.append('../../')
from resnet20 import BLOCKS_PER_GROUP, ResNet
import cifar100_resnet20_train  # import runa code that is necessary to run the model
from flax.traverse_util import flatten_dict
import numpy as np
from utils import flatten_params, unflatten_params
from weight_matching import resnet20_permutation_spec
from resnet20_torch import resnet20

2023-01-16 16:43:01.403972: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-01-16 16:43:02.352488: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-01-16 16:43:02.352591: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-01-16 16:43:02.352599: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.


In [68]:
model = ResNet(blocks_per_group=BLOCKS_PER_GROUP["resnet20"],
                   num_classes=512,
                   width_multiplier=4)

In [69]:
key1 , key2 = random.split(random.PRNGKey(0))
model_params = model.init(key2, jnp.zeros((1, 32, 32, 3)))
# from_bytes(model.init(random.PRNGKey(0), jnp.zeros((1, 32, 32, 3)))["params"], fh.read())

In [70]:

# def load_model(filepath):
#     with open(filepath, "rb") as fh:
#         return from_bytes(
#             model.init(random.PRNGKey(0), jnp.zeros((1, 32, 32, 3)))["params"], fh.read())
# model = load_model("../v7")
params = flatten_params(model_params)

In [71]:
spec = resnet20_permutation_spec().axes_to_perm
# this lines up the keys of corresponding dictionaries, at least it seems.

In [22]:
model1_save_path = "/srv/share/jbjorner3/checkpoints/REPAIR/resnet20x4_CIFAR50_clses[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49].pth.tar"
model2_save_path = "/srv/share/jbjorner3/checkpoints/REPAIR/resnet20x4_CIFAR50_clses[50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99].pth.tar"
# sd = torch.load('model_1.th')["state_dict"]
# sd = {k: v.cpu() for k, v in sd.items()}
# len(sd.keys())
DEVICE = "cuda"

model1 = resnet20(w=4).to(DEVICE)
sd = torch.load(model1_save_path, map_location=torch.device(DEVICE))
sd = {k: v.cpu() for k, v in sd.items()}
model1.load_state_dict(sd)

<All keys matched successfully>

In [23]:
list(sd.keys()).__len__()

128

In [32]:
import pdb

In [33]:
def torch_to_linen(torch_params, get_flax_keys):
    """Convert PyTorch parameters to Linen nested dictionaries"""

    def add_to_params(params_dict, nested_keys, param, is_conv=False):
        if len(nested_keys) == 1:
            key, = nested_keys
            try:
                params_dict[key] = np.transpose(param, (2, 3, 1, 0)) if is_conv else np.transpose(param)
            except:
                pdb.set_trace()
        else:
            assert len(nested_keys) > 1
            first_key = nested_keys[0]
            if first_key not in params_dict:
                params_dict[first_key] = {}
            add_to_params(params_dict[first_key], nested_keys[1:], param, ('conv' in first_key and \
                                                                         nested_keys[-1] != 'bias'))

    flax_params = {'params': {}, 'batch_stats': {}}
    for key, tensor in torch_params.items():
        flax_keys = get_flax_keys(key.split('.'))
        if flax_keys[-1] is not None:
            if flax_keys[-1] in ('mean', 'var'):
                add_to_params(flax_params['batch_stats'], flax_keys, tensor.detach().numpy())
            else:
                add_to_params(flax_params['params'], flax_keys, tensor.detach().numpy())

    return flax_params

In [35]:
list((v.shape, k) for k, v in sd.items())

[(torch.Size([64, 3, 3, 3]), 'conv1.weight'),
 (torch.Size([64]), 'bn1.weight'),
 (torch.Size([64]), 'bn1.bias'),
 (torch.Size([64]), 'bn1.running_mean'),
 (torch.Size([64]), 'bn1.running_var'),
 (torch.Size([]), 'bn1.num_batches_tracked'),
 (torch.Size([64, 64, 3, 3]), 'layer1.0.conv1.weight'),
 (torch.Size([64]), 'layer1.0.bn1.weight'),
 (torch.Size([64]), 'layer1.0.bn1.bias'),
 (torch.Size([64]), 'layer1.0.bn1.running_mean'),
 (torch.Size([64]), 'layer1.0.bn1.running_var'),
 (torch.Size([]), 'layer1.0.bn1.num_batches_tracked'),
 (torch.Size([64, 64, 3, 3]), 'layer1.0.conv2.weight'),
 (torch.Size([64]), 'layer1.0.bn2.weight'),
 (torch.Size([64]), 'layer1.0.bn2.bias'),
 (torch.Size([64]), 'layer1.0.bn2.running_mean'),
 (torch.Size([64]), 'layer1.0.bn2.running_var'),
 (torch.Size([]), 'layer1.0.bn2.num_batches_tracked'),
 (torch.Size([64, 64, 3, 3]), 'layer1.1.conv1.weight'),
 (torch.Size([64]), 'layer1.1.bn1.weight'),
 (torch.Size([64]), 'layer1.1.bn1.bias'),
 (torch.Size([64]), 'laye

In [36]:
len(sd), len(spec)

(128, 65)

In [37]:
dict(zip(sd.keys(), spec.keys()))

{'conv1.weight': 'conv1/kernel',
 'bn1.weight': 'norm1/scale',
 'bn1.bias': 'norm1/bias',
 'bn1.running_mean': 'blockgroups_0/blocks_0/conv1/kernel',
 'bn1.running_var': 'blockgroups_0/blocks_0/norm1/scale',
 'bn1.num_batches_tracked': 'blockgroups_0/blocks_0/norm1/bias',
 'layer1.0.conv1.weight': 'blockgroups_0/blocks_0/conv2/kernel',
 'layer1.0.bn1.weight': 'blockgroups_0/blocks_0/norm2/scale',
 'layer1.0.bn1.bias': 'blockgroups_0/blocks_0/norm2/bias',
 'layer1.0.bn1.running_mean': 'blockgroups_0/blocks_1/conv1/kernel',
 'layer1.0.bn1.running_var': 'blockgroups_0/blocks_1/norm1/scale',
 'layer1.0.bn1.num_batches_tracked': 'blockgroups_0/blocks_1/norm1/bias',
 'layer1.0.conv2.weight': 'blockgroups_0/blocks_1/conv2/kernel',
 'layer1.0.bn2.weight': 'blockgroups_0/blocks_1/norm2/scale',
 'layer1.0.bn2.bias': 'blockgroups_0/blocks_1/norm2/bias',
 'layer1.0.bn2.running_mean': 'blockgroups_0/blocks_2/conv1/kernel',
 'layer1.0.bn2.running_var': 'blockgroups_0/blocks_2/norm1/scale',
 'layer1.

In [51]:
import torch.nn as torch_nn
import torch.nn.functional as torch_F
import torch.nn.init as torch_init

def _weights_init(m):
    classname = m.__class__.__name__
    if isinstance(m, torch_nn.Linear) or isinstance(m, torch_nn.Conv2d):
        torch_init.kaiming_normal_(m.weight)

class torch_LambdaLayer(torch_nn.Module):
    def __init__(self, lambd):
        super(torch_LambdaLayer, self).__init__()
        self.lambd = lambd

    def forward(self, x):
        return self.lambd(x)


class torch_BasicBlock(torch_nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(torch_BasicBlock, self).__init__()
        self.conv1 = torch_nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = torch_nn.BatchNorm2d(planes)
        self.conv2 = torch_nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = torch_nn.BatchNorm2d(planes)

        self.shortcut = torch_nn.Sequential()
        if stride != 1 or in_planes != planes:
#             self.shortcut = LambdaLayer(lambda x:
#                                         F.pad(x[:, :, ::2, ::2], (0, 0, 0, 0, planes//4, planes//4), "constant", 0))
            self.shortcut = torch_nn.Sequential(
                torch_nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False),
                torch_nn.BatchNorm2d(planes)
            )


    def forward(self, x):
        out = torch_F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = torch_F.relu(out)
        return out


class torch_ResNet(torch_nn.Module):
    def __init__(self, block, num_blocks, w=1, num_classes=10):
        super(torch_ResNet, self).__init__()
        self.in_planes = w*16

        self.conv1 = torch_nn.Conv2d(3, w*16, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = torch_nn.BatchNorm2d(w*16)
        self.layer1 = self._make_layer(block, w*16, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, w*32, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, w*64, num_blocks[2], stride=2)
        self.linear = torch_nn.Linear(w*64, 512)

        self.apply(_weights_init)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion

        return torch_nn.Sequential(*layers)

    def forward(self, x):
        out = torch_F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = torch_F.avg_pool2d(out, out.size()[3])
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


def torch_resnet20(w=1):
    return torch_ResNet(torch_BasicBlock, [3, 3, 3], w=w)

In [52]:
from einops import reduce as jax_reduce
from flax import linen as jax_nn

def reverse_compose(x, fs):
  for f in fs:
    x = f(x)
  return x

class Block(jax_nn.Module):
  num_channels: int = None
  strides: int = None

  def setup(self):
    self.conv1 = jax_nn.Conv(features=self.num_channels,
                         kernel_size=(3, 3),
                         strides=self.strides,
                         use_bias=False)
    self.norm1 = jax_nn.BatchNorm(momentum=0.9, use_running_average=True)
    self.conv2 = jax_nn.Conv(features=self.num_channels, kernel_size=(3, 3), strides=1, use_bias=False)
    self.norm2 = jax_nn.BatchNorm(momentum=0.9, use_running_average=True)

    # When strides != 1, then it's 2, which means that we halve the width and height of the input, while doubling the
    # number of channels. Therefore we need to correspondingly halve the width and height of the residuals/shortcut.
    if self.strides != 1:
      assert self.strides == 2

      # Supposedly this is the original description, but it is not easily comaptible with our weight matching stuff
      # since it plays games with the channel structure by padding things around.
      # self.shortcut = lambda x: jnp.pad(x[:, ::2, ::2, :], (
      #     (0, 0), (0, 0), (0, 0), (self.num_channels // 4, self.num_channels // 4)),
      #                                   "constant",
      #                                   constant_values=0)

      # This is not the original, but is fairly common based on other implementations.
      self.shortcut = jax_nn.Sequential([
          jax_nn.Conv(features=self.num_channels,
                  kernel_size=(3, 3),
                  strides=self.strides,
                  use_bias=False),
          jax_nn.BatchNorm(momentum=0.9, use_running_average=True)
      ])
    else:
      self.shortcut = lambda x: x

  def __call__(self, x):
    y = x
    y = self.conv1(y)
    y = self.norm1(y)
    y = jax_nn.relu(y)
    y = self.conv2(y)
    y = self.norm2(y)
    return jax_nn.relu(y + self.shortcut(x))

class BlockGroup(jax_nn.Module):
  num_channels: int = None
  num_blocks: int = None
  strides: int = None

  def setup(self):
    assert self.num_blocks > 0
    self.blocks = (
        [Block(num_channels=self.num_channels, strides=self.strides)] +
        [Block(num_channels=self.num_channels, strides=1) for _ in range(self.num_blocks - 1)])

  def __call__(self, x):
    return reverse_compose(x, self.blocks)

class ResNet(jax_nn.Module):
  blocks_per_group: int = None
  num_classes: int = None
  width_multiplier: int = 1

  def setup(self):
    wm = self.width_multiplier

    self.conv1 = jax_nn.Conv(features=16 * wm, kernel_size=(3, 3), use_bias=False)
    self.norm1 = jax_nn.BatchNorm(momentum=0.9, use_running_average=True)

    channels_per_group = (16 * wm, 32 * wm, 64 * wm)
    strides_per_group = (1, 2, 2)
    self.blockgroups = [
        BlockGroup(num_channels=c, num_blocks=b, strides=s)
        for c, b, s in zip(channels_per_group, self.blocks_per_group, strides_per_group)
    ]

    self.dense = jax_nn.Dense(self.num_classes)

  def __call__(self, x):
    x = self.conv1(x)
    x = self.norm1(x)
    x = jax_nn.relu(x)
    x = reverse_compose(x, self.blockgroups)
    x = jax_reduce(x, "n h w c -> n c", "mean")
    x = self.dense(x)
    # x = jax_nn.log_softmax(x)
    return x

def jax_resnet20(w=1):
  return ResNet(BLOCKS_PER_GROUP["resnet20"], num_classes=512, width_multiplier=w)
BLOCKS_PER_GROUP = {
    "resnet20": (3, 3, 3),
    "resnet32": (5, 5, 5),
    "resnet44": (7, 7, 7),
    "resnet56": (9, 9, 9),
    "resnet110": (18, 18, 18),
}


In [53]:
from jax import random as jax_random
import jax
import re

In [54]:
model1_save_path = "/srv/share/jbjorner3/checkpoints/REPAIR/resnet20x4_CIFAR50_clses[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49].pth.tar"
# model2_save_path = "/srv/share/jbjorner3/checkpoints/REPAIR/resnet20x4_CIFAR50_clses[50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99].pth.tar"
# sd = torch.load('model_1.th')["state_dict"]
# sd = {k: v.cpu() for k, v in sd.items()}
# len(sd.keys())
DEVICE = "cuda"

torch_model = resnet20(w=4).to(DEVICE)
sd = torch.load(model1_save_path, map_location=torch.device(DEVICE))
sd = {k: v.cpu() for k, v in sd.items()}
torch_model.load_state_dict(sd)

<All keys matched successfully>

In [64]:
# init torch_resnet20
# print(torch_model)
# print(dict(torch_model.state_dict()))
torch_dict_sizes = jax.tree_util.tree_map(lambda x: list(x.shape), dict(torch_model.state_dict()))
def not_bn_stat(x):
    return "num_batches_tracked" not in x and 'mean' not in x and 'var' not in x
# torch_list_sizes = list(item for item in torch_dict_sizes.items() if not not_bn_stat(item[0]) and "num_batches" not in item[0])
torch_list_sizes = list(item for item in torch_dict_sizes.items() if "num_batches" not in item[0])
print(torch_list_sizes.__len__())
torch_list_sizes

107


[('bn1.bias', [64]),
 ('bn1.running_mean', [64]),
 ('bn1.running_var', [64]),
 ('bn1.weight', [64]),
 ('conv1.weight', [64, 3, 3, 3]),
 ('layer1.0.bn1.bias', [64]),
 ('layer1.0.bn1.running_mean', [64]),
 ('layer1.0.bn1.running_var', [64]),
 ('layer1.0.bn1.weight', [64]),
 ('layer1.0.bn2.bias', [64]),
 ('layer1.0.bn2.running_mean', [64]),
 ('layer1.0.bn2.running_var', [64]),
 ('layer1.0.bn2.weight', [64]),
 ('layer1.0.conv1.weight', [64, 64, 3, 3]),
 ('layer1.0.conv2.weight', [64, 64, 3, 3]),
 ('layer1.1.bn1.bias', [64]),
 ('layer1.1.bn1.running_mean', [64]),
 ('layer1.1.bn1.running_var', [64]),
 ('layer1.1.bn1.weight', [64]),
 ('layer1.1.bn2.bias', [64]),
 ('layer1.1.bn2.running_mean', [64]),
 ('layer1.1.bn2.running_var', [64]),
 ('layer1.1.bn2.weight', [64]),
 ('layer1.1.conv1.weight', [64, 64, 3, 3]),
 ('layer1.1.conv2.weight', [64, 64, 3, 3]),
 ('layer1.2.bn1.bias', [64]),
 ('layer1.2.bn1.running_mean', [64]),
 ('layer1.2.bn1.running_var', [64]),
 ('layer1.2.bn1.weight', [64]),
 ('l

In [56]:
torch_state_dict = dict(torch_model.state_dict())

def fix_keys(old_key):
    new_key = old_key
    substitutions =[
        ("bn", "norm"),
        ("layer", "blockgroups_"),
        ("running_mean", "mean"),
        ("running_var", "var"),
        ("weight", "kernel")
    ]
    for sub in substitutions:
        new_key = new_key.replace(sub[0], sub[1])
    new_key = re.sub(r"blockgroups_(\d)", lambda x: f"blockgroups_{int(x.group(1))-1}", new_key)
    new_key = re.sub(r"blockgroups_(\d)\.(\d)\.", "blockgroups_\g<1>.blocks_\g<2>.", new_key)
    new_key = re.sub(r"shortcut\.", "shortcut.layers_", new_key)
    new_key = re.sub(r"norm(\d).kernel", "norm\g<1>.scale", new_key)
    return new_key
def expand_dict(parent_dict, key, value):
    # expand dict along periods of the key
    keys = key.split(".")
    if 'shortcut' in keys and 'layers_1' in keys and 'kernel' in keys:
        keys[-1] = 'scale'
    if 'linear' in keys:
        keys[keys.index('linear')] = 'dense'
    curr_dict = parent_dict
    for new_key in keys[:-1]:
        if curr_dict.get(new_key, None) == None:
            curr_dict[new_key] = dict()
        curr_dict = curr_dict[new_key]    
    
    curr_dict[keys[-1]] = value

def fix_vals(old_key, old_val):
    new_val = old_val.detach().cpu().numpy()
    if "conv" in old_key or 'shortcut.0' in old_key:
        new_val = jnp.transpose(new_val, (2, 3, 1, 0))
    elif 'linear.weight' in old_key:
        new_val = jnp.transpose(new_val, (1, 0))
    return new_val
        
torch_state_dict = {fix_keys(k): fix_vals(k, v) for k, v in torch_state_dict.items()}

torch_batch_stats = {k: v for k, v in torch_state_dict.items() if not not_bn_stat(k) and "num_batches" not in k}
torch_params = {k: v for k, v in torch_state_dict.items() if not_bn_stat(k)}
torch_batch_stats_expanded = dict()
torch_params_expanded = dict()
for k, v in torch_batch_stats.items():
    expand_dict(torch_batch_stats_expanded, k, v)
for k, v in torch_params.items():
    expand_dict(torch_params_expanded, k, v)
torch_params_dict = {
        "batch_stats": torch_batch_stats_expanded, 
        "params": torch_params_expanded
    }
from pprint import pprint
pprint(jax.tree_util.tree_map(lambda x: x.shape, torch_params_dict))
# translate the shapes of the tensors to match with jax's formatting: https://flax.readthedocs.io/en/latest/advanced_topics/convert_pytorch_to_flax.html
# model.apply(torch_params_dict, x)

{'batch_stats': {'blockgroups_0': {'blocks_0': {'norm1': {'mean': (64,),
                                                          'var': (64,)},
                                                'norm2': {'mean': (64,),
                                                          'var': (64,)}},
                                   'blocks_1': {'norm1': {'mean': (64,),
                                                          'var': (64,)},
                                                'norm2': {'mean': (64,),
                                                          'var': (64,)}},
                                   'blocks_2': {'norm1': {'mean': (64,),
                                                          'var': (64,)},
                                                'norm2': {'mean': (64,),
                                                          'var': (64,)}}},
                 'blockgroups_1': {'blocks_0': {'norm1': {'mean': (128,),
                                              

In [40]:
# translate the shapes of the tensors to match with jax's formatting: https://flax.readthedocs.io/en/latest/advanced_topics/convert_pytorch_to_flax.html
# out = model.apply(torch_params_dict, x)
# out.shape

In [57]:
# init jax_resnet20
key1, key2 = jax_random.split(jax_random.PRNGKey(0), 2)
x = jax_random.uniform(key1, (1, 32, 32, 3))

jax_model = jax_resnet20(w=4)
jax_params = jax_model.init(key2, x)
y = jax_model.apply(jax_params, x)

In [83]:
jax_params.keys()

frozen_dict_keys(['params', 'batch_stats'])

In [101]:
jax_params['params']['conv1']['kernel'].shape

(3, 3, 3, 64)

In [59]:
jax_params_dict = jax.tree_util.tree_map(lambda x: jax.numpy.array(x.shape),jax_params)
count = 0
def incrment_identity(x):
    global count
    count += 1
    return x
jax_params_dict = jax.tree_util.tree_map(incrment_identity, jax_params_dict)
print(count)# __repr__()[:1000])
jax_params_dict

107


FrozenDict({
    batch_stats: {
        blockgroups_0: {
            blocks_0: {
                norm1: {
                    mean: DeviceArray([64], dtype=int32),
                    var: DeviceArray([64], dtype=int32),
                },
                norm2: {
                    mean: DeviceArray([64], dtype=int32),
                    var: DeviceArray([64], dtype=int32),
                },
            },
            blocks_1: {
                norm1: {
                    mean: DeviceArray([64], dtype=int32),
                    var: DeviceArray([64], dtype=int32),
                },
                norm2: {
                    mean: DeviceArray([64], dtype=int32),
                    var: DeviceArray([64], dtype=int32),
                },
            },
            blocks_2: {
                norm1: {
                    mean: DeviceArray([64], dtype=int32),
                    var: DeviceArray([64], dtype=int32),
                },
                norm2: {
                    m

In [19]:
import flax

In [112]:
import pickle
jax_params_permuted = pickle.load(open("/srv/share/gstoica3/checkpoints/REPAIR/flax_cifar50_2_permuted_to_1.pkl","rb"))

In [119]:
# convert back to pytorch from jax.
def collapse_dict(jax_params_dict):
    torch_params_dict = dict()
    
    for old_dict in [jax_params_dict]:
        # print(old_dict)
        recursively_build_dict([], old_dict, torch_params_dict)
    # pprint(torch_params_dict)
    torch_params_dict = {fix_keys(k): v for k, v in torch_params_dict.items()}
    torch_params_dict = {k: fix_vals(k, v) for k, v in torch_params_dict.items()}

    return torch_params_dict
def recursively_build_dict(old_keys, old_dict, new_dict):
    if isinstance(old_dict, flax.core.frozen_dict.FrozenDict):
        for old_key, old_val in old_dict.items():
            recursively_build_dict(old_keys + [old_key], old_val, new_dict)
    else:
        # now we have an array to convert
        new_dict[".".join(old_keys)] = old_dict
def fix_keys(old_key):
    new_key = old_key
    new_key = re.sub(r"norm(\d).scale", "norm\g<1>.kernel", new_key)
    new_key = re.sub(r"shortcut\.layers_", "shortcut.", new_key)
    new_key = re.sub(r"shortcut\.1\.scale", "shortcut.1.weight", new_key)
    new_key = re.sub(r"blockgroups_(\d)\.blocks_(\d)\.", "blockgroups_\g<1>.\g<2>.", new_key)
    new_key = re.sub(r"blockgroups_(\d)", lambda x: f"blockgroups_{int(x.group(1))+1}", new_key)
    substitutions =[("bn", "norm"),("layer", "blockgroups_"),("running_mean", "mean"),("running_var", "var"),("weight", "kernel"),("linear","dense")]
    for sub in substitutions[::-1]:
        new_key = new_key.replace(sub[1], sub[0]) # in reverse order of old fix_keys
    return new_key
def fix_vals(old_key, old_val):
    new_val = old_val
    if "conv" in old_key or 'shortcut.0' in old_key:
        # new_val = jnp.transpose(new_val, (2, 3, 1, 0))
        new_val = jnp.transpose(new_val, (3, 2, 0, 1))
    elif 'linear.weight' in old_key:
        new_val = jnp.transpose(new_val, (1, 0))
    new_val = torch.tensor(np.array(new_val))
    
    return new_val

# def find_permutation(bn_bias_permuted, bn_bias_not_permuted):
#     # find the picking order necessary to index the not permuted to achieve the permuted.
#     abs_dif_matrix = np.abs(bn_bias_permuted[None, :] - bn_bias_not_permuted[:, None])
#     return np.argmin(abs_dif_matrix, axis=0)
# def fix_batch_stats_permutations(new_torch_params_dict, old_torch_params_dict):
#     # correct the batch stats based on the permutations in the weight and bias (should be the same?)
#     def fix_batch_stats_val(k, v):
#         if "bn" in k or "shortcut.1" in k:
#             print(k)
#             if "running_mean" in k or "running_var" in k:
#                 bn_mean_key = k.replace("running_mean", "bias").replace("running_var", "bias")
#                 perm_bn_mean = new_torch_params_dict[bn_mean_key]
#                 not_perm_bn_mean = old_torch_params_dict[bn_mean_key]
#                 perm_indexer = find_permutation(perm_bn_mean, not_perm_bn_mean)
#                 return
#             # remove the word bias or weight, and replace with running mean and running var to get access to these
#         return v
#     fixed_torch_params_dict = {k: v for k, v in new_torch_params_dict.items()}


output_dict = collapse_dict(jax_params_permuted)
print(len(output_dict))
# to test set equality both ways.
# pprint({item[0] for item in torch_list_sizes}.difference(output_dict.keys()))
# pprint(set(output_dict.keys()).difference({item[0] for item in torch_list_sizes}))
# pprint(jax.tree_util.tree_map(lambda x: x.shape, output_dict))
# pprint()
# correct_sizes_dict = {k:torch.Size(v) for k, v in torch_list_sizes}
# for key, val in output_dict.items():
#     if correct_sizes_dict[key] != val.shape:
#         print("incorrect matching", key, val.shape, correct_sizes_dict[key])
#         print(correct_sizes_dict[key])
#         print(val.shape)
#         print(fix_vals(key, val.detach().numpy()).shape)
# find and apply batch stats permutation based on the permutation of the bn bias.
# jax_params.keys()
output_dict
# torch_list_sizes
# old_torch_params_dict = dict(torch_model.state_dict())
pickle.dump(output_dict, open("/srv/share/jbjorner3/checkpoints/REPAIR/pytorch_cifar50_2_permuted_to_1_jakob.pkl", "wb"))

107


In [117]:
torch_resnet20(w=4).load_state_dict(output_dict)

<All keys matched successfully>